In [ ]:
from google.colab import files
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/mono_16000_voice.zip -d /content/

Archive:  /content/gdrive/MyDrive/mono_16000_voice.zip
  inflating: /content/mono_16000_voice/0.1.wav  
  inflating: /content/mono_16000_voice/1.1.wav  
  inflating: /content/mono_16000_voice/10.1.wav  
  inflating: /content/mono_16000_voice/100.1.wav  
  inflating: /content/mono_16000_voice/1000.1.wav  
  inflating: /content/mono_16000_voice/1001.1.wav  
  inflating: /content/mono_16000_voice/1002.1.wav  
  inflating: /content/mono_16000_voice/1003.1.wav  
  inflating: /content/mono_16000_voice/1004.1.wav  
  inflating: /content/mono_16000_voice/1005.1.wav  
  inflating: /content/mono_16000_voice/1005.2.wav  
  inflating: /content/mono_16000_voice/1006.1.wav  
  inflating: /content/mono_16000_voice/1006.2.wav  
  inflating: /content/mono_16000_voice/1007.1.wav  
  inflating: /content/mono_16000_voice/1008.1.wav  
  inflating: /content/mono_16000_voice/1008.2.wav  
  inflating: /content/mono_16000_voice/1009.1.wav  
  inflating: /content/mono_16000_voice/101.1.wav  
  inflating: /conte

In [ ]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.6 MB/s eta 0:00:00


In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from jiwer import wer, cer

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset

In [ ]:
# Словарь для кодирования каждого символа
char_map = {"": 0, "а": 1, "б": 2, "в": 3, "г": 4, "д": 5, "е": 6, "ё": 7, "ж": 8, "з": 9, "и": 10, "й": 11,
            "к": 12, "л": 13, "м": 14, "н": 15, "о": 16, "п": 17, "р": 18, "с": 19, "т": 20, "у": 21,
            "ф": 22, "х": 23, "ч": 24, "ц": 25, "ш": 26, "щ": 27, "ъ": 28, "ы": 29, "ь": 30, "э": 31, "ю": 32,
            "я": 33, " ": 34}

# Обратный словарь для раскодирования символов
index_map = dict()
for key, value in char_map.items():
    index_map[value] = key

In [ ]:
# Функция для кодирования каждого символа в тексте
encode_characte = lambda text: [char_map[char] for char in text]

# Функция для декодирования каждой метки
decode_characte = lambda labels: "".join([index_map[label] for label in labels])

In [ ]:
# Функция для обработки пакета данных
def data_processing(data):
    spectrograms = list()
    labels = list()
    input_lengths = list()
    label_lengths = list()

    for (waveform, utterance) in data:
        # mfccs = librosa.feature.mfcc(y=waveform, sr=16000, n_mfcc=30, hop_length=640)
        # mfccs = librosa.feature.melspectrogram(y=waveform, sr=16000, n_mels=128, hop_length=640)
        spectrogram = torch.FloatTensor(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spectrogram)
        label = torch.Tensor(encode_characte(utterance.lower()))
        labels.append(label)
        input_lengths.append(spectrogram.shape[0])
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)
    return spectrograms, labels, input_lengths, label_lengths

In [ ]:
# Функция для нахождения нужных меток в тензоре, который возвращает нейросеть
def GreedyDecoder(output, labels, label_lengths, blank_label=0, collapse_repeated=True):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(decode_characte(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(decode_characte(decode))
    return decodes, targets

In [ ]:
# # Функция для загрузки датасета
# def read_files(exel_file, wavs_dir):
#     file = pd.read_excel(exel_file)
#     y = [sentence for sentence in file['Русская речь']]
#     files_in_dir = os.listdir(wavs_dir)
#     X = list()

#     for i in range(1, 2001):
#         file_name = f'{i}.wav'
#         sample = librosa.load(wavs_dir + file_name, sr=16000)[0]
#         mfccs = librosa.feature.mfcc(y=sample, sr=16000, n_mfcc=30, hop_length=640)
#         # X.append(sample)
#         X.append(mfccs)

#     return X, y

In [ ]:
# Функция для загрузки датасета
def read_files(exel_file, wavs_dir):
    file = pd.read_excel(exel_file)
    old_id = [int(sentence) for sentence in file['old_id']]
    text = [sentence for sentence in file['text']]
    files_in_dir = os.listdir(wavs_dir)
    X = list()
    y = list()

    for wav in files_in_dir:
        id = int(wav.split('.')[0])
        index = old_id.index(id)
        wav_text = text[index]
        sample = librosa.load(wavs_dir + wav, sr=16000)[0]
        mfccs = librosa.feature.mfcc(y=sample, sr=16000, n_mfcc=30, hop_length=640)
        X.append(mfccs)
        y.append(wav_text)

    print("Файлы прочитаны!")

    return X, y

In [ ]:
X, y = read_files("/content/Speeches.xlsx", "/content/mono_16000_voice/")

Файлы прочитаны!


In [ ]:
# Функция удаляет ненужные символы
def remove_characters(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace("-", " ")
    sentence = ''.join(filter(lambda x: x in char_map, sentence))
    sentence = " ".join(sentence.split())
    return sentence

y = list(map(remove_characters, y))

In [ ]:
# Разделяем данные на обучающие и тестовые
n = int(len(X) // 1.11)
X_train = X[:n]
X_test = X[n:]
y_train = y[:n]
y_test = y[n:]

In [ ]:
len(X_train), len(y_train)

(2519, 2519)

In [ ]:
# Класс отвечающий за формирование набора данных
class AudioDataset(Dataset):
    def __init__(self, audio_list, text_list):
        self.audio_list = audio_list
        self.text_list = text_list

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, index):
        audio = self.audio_list[index]
        text = self.text_list[index]
        return audio, text

In [ ]:
# В данной ячейки представлены классы-обертки для стандартных классов PyTorch.
# Это нужно для того, чтобы легко помещать их в Sequential.

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1,
                 batch_first=False, dropout=0, bidirectional=False):
        super(GRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers,
                          batch_first=batch_first, dropout=dropout,
                          bidirectional=bidirectional)

    def forward(self, x):
        x, _ = self.gru(x)
        return x

class BiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1,
                 batch_first=False, dropout=0):
        super(BiGRU, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers,
                          batch_first=batch_first, dropout=dropout,
                          bidirectional=True)

    def forward(self, x):
        x, _ = self.gru(x)
        x = x[:,:, self.hidden_size:] + x[:,:,:self.hidden_size]
        return x

class BackGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1,
                 batch_first=False, dropout=0):
        super(BackGRU, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers,
                          batch_first=batch_first, dropout=dropout)

    def forward(self, x):
        reverse_x = torch.flip(x, [1])
        reverse_x, _ = self.gru(reverse_x)
        return reverse_x

class Convolution1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(Convolution1d, self).__init__()
        self.conv1d = nn.Conv1d(in_channels, out_channels,
                                kernel_size=kernel_size, stride=stride,
                                padding=padding)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv1d(x)
        x = x.transpose(1, 2)
        return x

class BiConvolution1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(BiConvolution1d, self).__init__()
        self.conv1d1 = nn.Conv1d(in_channels, out_channels,
                                kernel_size=kernel_size, stride=stride,
                                padding=padding)
        self.conv1d2 = nn.Conv1d(in_channels, out_channels,
                                kernel_size=kernel_size, stride=stride,
                                padding=padding)

    def forward(self, x):
        x = x.transpose(1, 2)
        reverse_x = torch.flip(x, [2])
        x = self.conv1d1(x)
        reverse_x = self.conv1d2(reverse_x)
        x = x + reverse_x
        x = x.transpose(1, 2)
        return x

class Activation(nn.Module):
    def __init__(self, func="relu"):
        super(Activation, self).__init__()
        if func == "relu":
            self.nonlinearity = nn.ReLU()
        elif func == "leakyrelu":
            self.nonlinearity = nn.LeakyReLU()
        elif func == "elu":
            self.nonlinearity = nn.ELU()
        elif func == "selu":
            self.nonlinearity = nn.SELU()
        else:
            self.nonlinearity = nn.GELU()

    def forward(self, x):
        x = self.nonlinearity(x)
        return x

class LayerNormalization(nn.Module):
    def __init__(self, size):
        super(LayerNormalization, self).__init__()
        self.norm = nn.LayerNorm(size)

    def forward(self, x):
        x = self.norm(x)
        return x

class Dropout(nn.Module):
    def __init__(self, dropout):
        super(Dropout, self).__init__()
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        x = self.drop(x)
        return x

class Dense(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0):
        super(Dense, self).__init__()
        self.linear = nn.Linear(input_size, hidden_size)

    def forward(self, x):
        x = self.linear(x)
        return x

class Dense2(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0):
        super(Dense2, self).__init__()
        self.linear = nn.Linear(input_size, hidden_size)

    def forward(self, x):
        x = self.linear(x)
        x = F.relu(x)
        x = F.hardtanh(x, 0, 20)
        return x

In [ ]:
class SpeechModel(nn.Module):
    def __init__(self):
        super(SpeechModel, self).__init__()
        self.first_layer = nn.Sequential(
            Convolution1d(30, 256, 3, 1, 1),
            LayerNormalization(256),
            Activation(),
            GRU(256, 256, 1, True, 0),
            Dense(256, 256)
        )

        self.second_layer = nn.Sequential(
            Convolution1d(30, 256, 3, 1, 1),
            LayerNormalization(256),
            Activation(),
            GRU(256, 256, 1, True, 0),
            Dense(256, 256)
        )

        self.third_block = nn.Sequential(
            LayerNormalization(256),
            Convolution1d(256, 256, 3, 1, 1),
            LayerNormalization(256),
            Activation(),
            BiGRU(256, 256, 1, True, 0),
            Dense(256, 256)
        )

        self.linear = nn.Sequential(
            Dense(256, 35),
            nn.LogSoftmax(2)
        )

    def forward(self, x):
        reverse_x = torch.flip(x, [1])
        x = self.first_layer(x)
        reverse_x = self.second_layer(reverse_x)
        x = x + reverse_x
        x = self.third_block(x)
        x = self.linear(x)
        return x

In [ ]:
# Функция, нужная для обучения модели
def train(model, device, train_loader, criterion, optimizer, scheduler, epoch):
    model.train()
    train_loss = 0
    train_cer, train_wer = [], []
    data_len = len(train_loader.dataset)
    for batch_idx, _data in enumerate(train_loader):
        spectrograms, labels, input_lengths, label_lengths = _data
        spectrograms, labels = spectrograms.to(device), labels.to(device)

        optimizer.zero_grad()

        output = model(spectrograms)
        # input_lengths = [output.shape[1]]
        output = output.transpose(0, 1)

        loss = criterion(output, labels, input_lengths, label_lengths)
        train_loss += loss.item() / len(train_loader)
        loss.backward()

        optimizer.step()
        scheduler.step()
        if (batch_idx % 20 == 0 and batch_idx != 0) or batch_idx == data_len:
            print('Обучающие данные: эпоха {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(spectrograms), data_len,
                100. * batch_idx / len(train_loader), loss.item()))


# Функция, нужная для промежуточного тестирования модели
def test(model, device, test_loader, criterion, epoch):
    print('\nПодсчёт...')
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []
    with torch.no_grad():
        for i, _data in enumerate(test_loader):
            spectrograms, labels, input_lengths, label_lengths = _data
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            output = model(spectrograms)  # (batch, time, n_class)
            # input_lengths = [output.shape[1]]
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)

            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
            for j in range(len(decoded_preds)):
                test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                test_wer.append(wer(decoded_targets[j], decoded_preds[j]))

    avg_cer = sum(test_cer)/len(test_cer)
    avg_wer = sum(test_wer)/len(test_wer)

    print('Тестовые данные: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n'.format(test_loss, avg_cer, avg_wer))

In [ ]:
# Функция, которая настраивает и запускает обучение модели
def main(learning_rate=5e-4, batch_size=20, epochs=10):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    train_dataset = AudioDataset(X_train, y_train)
    test_dataset = AudioDataset(X_test, y_test)

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = data.DataLoader(dataset=train_dataset,
                                batch_size=batch_size,
                                shuffle=True,
                                collate_fn=lambda x: data_processing(x),
                                **kwargs)
    test_loader = data.DataLoader(dataset=test_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                collate_fn=lambda x: data_processing(x),
                                **kwargs)

    n = 256
    m = n * 2

    model = nn.Sequential(
        Convolution1d(30, n, 3, 1, 1),
        LayerNormalization(n),
        Activation(),
        GRU(n, n, 1, True, 0, True),
        Dense(m, m),
        LayerNormalization(m),
        Activation(),

        Convolution1d(m, m, 3, 1, 1),
        LayerNormalization(m),
        Activation(),
        GRU(m, n, 1, True, 0, True),
        Dense(m, m),
        LayerNormalization(m),
        Activation(),

        Convolution1d(m, m, 3, 1, 1),
        LayerNormalization(m),
        Activation(),
        GRU(m, n, 1, True, 0, True),
        Dense(m, m),
        LayerNormalization(m),
        Activation(),

        Convolution1d(m, m, 3, 1, 1),
        LayerNormalization(m),
        Activation(),
        GRU(m, n, 1, True, 0, True),
        Dense(m, m),
        LayerNormalization(m),
        Activation(),

        Dense(m, 35),
        nn.LogSoftmax(dim=2)
    ).to(device)

    # model = SpeechModel().to(device)

    print(model)
    print('Количество параметров модели:', sum([param.nelement() for param in model.parameters()]))
    print()

    optimizer = optim.Adam(model.parameters(), learning_rate)
    criterion = nn.CTCLoss(blank=0).to(device)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate,
                                            steps_per_epoch=int(len(train_loader)),
                                            epochs=epochs,
                                            anneal_strategy='linear')

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, criterion, optimizer, scheduler, epoch)
        test(model, device, test_loader, criterion, epoch)

    # torch.save(model, '/content/model.pt')
    torch.save(model.state_dict(), '/content/model.pth')
    return model, device

In [ ]:
%%time
learning_rate = 0.0005
batch_size = 5
epochs = 25

model, device = main(learning_rate, batch_size, epochs)

Sequential(
  (0): Convolution1d(
    (conv1d): Conv1d(30, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  )
  (1): LayerNormalization(
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (2): Activation(
    (nonlinearity): ReLU()
  )
  (3): GRU(
    (gru): GRU(256, 256, batch_first=True, bidirectional=True)
  )
  (4): Dense(
    (linear): Linear(in_features=512, out_features=512, bias=True)
  )
  (5): LayerNormalization(
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (6): Activation(
    (nonlinearity): ReLU()
  )
  (7): Convolution1d(
    (conv1d): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  )
  (8): LayerNormalization(
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (9): Activation(
    (nonlinearity): ReLU()
  )
  (10): GRU(
    (gru): GRU(512, 256, batch_first=True, bidirectional=True)
  )
  (11): Dense(
    (linear): Linear(in_features=512, out_features=512, bias=True)
  )
  (12): LayerN

In [ ]:
device = torch.device("cpu")
model.to(device)
model.eval()
spectro = []

index = 4

print(X_test[index].size)
print(y_test[index])

spectrogram_tensor = torch.FloatTensor(X_test[index]).squeeze(0).transpose(0, 1).unsqueeze(0)

with torch.no_grad():
    spectrogram_tensor.to(device)
    output = model(spectrogram_tensor)
    print(output.size())

    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    for i, args in enumerate(arg_maxes):
        decode = []
        for j, index in enumerate(args):
            if index != 0:
                if True and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(decode_characte(decode))

print(decodes[0])

3360
не выходи из комнаты
torch.Size([1, 112, 35])
не выходе из комнаты


In [ ]:
# Функция переводит аудиофайл в текст
def predict(model, file_name, device):
    model.to(device)
    model.eval()
    spectro = []

    sampl = librosa.load(file_name, sr=16000)[0]
    mfccs = librosa.feature.mfcc(y=sampl, sr=16000, n_mfcc=30, hop_length=640, n_fft=2048)
    spectrogram_tensor = torch.FloatTensor(mfccs).squeeze(0).transpose(0, 1).unsqueeze(0)
    print(spectrogram_tensor.size())

    with torch.no_grad():
        spectrogram_tensor.to(device)
        output = model(spectrogram_tensor)  # (batch, time, n_class)
        print(output.size())

        arg_maxes = torch.argmax(output, dim=2)
        decodes = []
        for i, args in enumerate(arg_maxes):
            decode = []
            for j, index in enumerate(args):
                if index != 0:
                    if True and j != 0 and index == args[j -1]:
                        continue
                    decode.append(index.item())
            decodes.append(decode_characte(decode))

    return decodes[0]

In [ ]:
device = torch.device("cpu")
predict(model, '/content/mono_voice/1801.wav', device)

torch.Size([1, 265, 30])
torch.Size([1, 265, 35])


'могулия попрасить вачсавет поко вою этова пробля ного пороса'

In [ ]:
sample = librosa.load('/content/mono_voice/1.wav', sr=16000)[0]

In [ ]:
sample.shape

(79872,)

In [ ]:
mfccs = librosa.feature.mfcc(y=sample, sr=16000, n_mfcc=30, hop_length=640)

In [ ]:
mfccs.shape

(30, 125)

In [ ]:
mfccs.T[100]

array([-4.7254480e+02,  3.0608105e+01, -8.6714535e+00,  1.2766712e+00,
        5.8672104e+00, -3.6032848e+00, -8.1073177e-01, -1.7650914e+00,
       -2.3796582e+00,  3.1737804e+00,  6.7866832e-01, -3.6975060e+00,
       -4.2084527e-01, -1.7753491e+00, -4.3187003e+00, -4.1776638e+00,
       -2.9993851e+00, -5.8667243e-02,  2.0077569e+00,  8.3077770e-01,
        2.6146169e+00,  1.4099152e+00, -2.4030864e+00, -2.5973098e+00,
       -1.9240938e+00, -2.1044173e+00,  1.0937614e+00, -2.4209732e-01,
       -4.6541624e+00, -5.2274690e+00], dtype=float32)